# Imports

In [1]:
import torch
import torch.nn as nn
from models import EncoderRNN, LuongAttnDecoderRNN, GreedySearchDecoder, BeamSearchDecoder
from vocabulary import Voc
from utils import evaluate, normalizeString, calculate_distinct_n_grams
import matplotlib.pyplot as plt
import numpy as np

# Load Model

In [2]:
# Specify the path to the checkpoint file
checkpoint_path = 'checkpoints/4000_checkpoint.tar'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

# Access the information from the checkpoint
iteration = checkpoint['iteration']
encoder_state_dict = checkpoint['en']
decoder_state_dict = checkpoint['de']
encoder_optimizer_state_dict = checkpoint['en_opt']
decoder_optimizer_state_dict = checkpoint['de_opt']
train_loss = checkpoint['train loss']
val_loss = checkpoint['val loss']
train_perplexity = checkpoint['train perplexity']
val_perplexity = checkpoint['val perplexity']
voc_dict = checkpoint['voc_dict']

In [3]:
# Reconstruct vocabulary
voc = Voc(voc_dict['name'])
voc.__dict__ = voc_dict

In [4]:
# Reconstruct models
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1

encoder = EncoderRNN(hidden_size=hidden_size, 
                     embedding=nn.Embedding(num_embeddings=voc.num_words, embedding_dim=hidden_size),
                     n_layers=encoder_n_layers, 
                     dropout=dropout)

decoder = LuongAttnDecoderRNN(attn_model='dot', 
                              embedding=nn.Embedding(num_embeddings=voc.num_words, embedding_dim=hidden_size), 
                              hidden_size=hidden_size, 
                              output_size=voc.num_words, 
                              n_layers=decoder_n_layers, 
                              dropout=dropout)

# Load state into encoder and decoder
encoder.load_state_dict(encoder_state_dict)
decoder.load_state_dict(decoder_state_dict)

# Set layers to eval mode
encoder.eval()
decoder.eval()

LuongAttnDecoderRNN(
  (embedding): Embedding(8901, 500)
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(500, 500, num_layers=2, dropout=0.1)
  (concat): Linear(in_features=1000, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=8901, bias=True)
  (attn): Attn()
)

# Loss & Perplexity

In [ ]:
plt.plot(train_loss, label='Train loss')
plt.plot(val_loss, label='Validation loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_perplexity, label='Train perplexity')
plt.plot(val_perplexity, label='Validation perplexity')
plt.xlabel('Iterations')
plt.ylabel('Perplexity')
plt.legend()
plt.show()

# Example Inputs

In [5]:
# Input utterances generated by chat-GPT
input_utts = [
    "Why do I always get stuck in the middle of everyone's problems ?",
    "Imagine if we could solve disputes with a dance-off instead of arguments .",
    "Do you think a penguin could be a good pet in the city ?",
    "I accidentally sent a love confession to the group chat , help !",
    "What's the weirdest thing you've ever done to impress a crush ?",
    "Have you ever walked into a room and forgotten why you're there ?",
    "Why is it that every time I try to cook , the fire alarm goes off ?",
    "I just tried talking to my crush , and I could only speak in movie quotes .",
    "If you had to choose , would you rather give up coffee or pizza ?",
    "Is it okay to wear socks with sandals if nobody's going to see ?",
    "Why do I feel like my laundry multiplies every time I blink ?",
    "Do you believe in love at first sight , or should I walk by again ?",
    "What's your take on eating pizza with a fork and knife ?",
    "Is talking to plants really supposed to help them grow ?",
    "I dreamed I was a muffin ; what do you think it means ?",
    "Why is it that I can never find my keys when I'm late ?",
    "Do you think ducks have best friends ?",
    "I just found out I've been using a word wrong my entire life .",
    "Would you rather fight one horse-sized duck or a hundred duck-sized horses ?",
    "What's the protocol for accidentally waving at someone who wasn't waving at you ?",
    "Do you ever practice conversations in your head before having them ?",
    "How many days in a row can you wear the same pair of jeans ?",
    "Why does my phone always die at the worst possible times ?",
    "Have you ever sent a text about someone to that person by accident ?",
    "What's the most embarrassing song you love ?",
    "Why do I feel like I need a vacation from my vacation ?",
    "Do you think it's possible to be too good at thumb wrestling ?",
    "What would you do if you found out you had a secret twin ?",
    "Why are my awkward moments always witnessed by the most people ?",
    "If you could have any superpower , but it had to be really mundane , what would it be ?",
    "What's the silliest reason you've ever had a falling out with someone ?",
    "Have you ever tried to bake something and it turned into a disaster ?",
    "Is double-dipping really that bad ?",
    "What's the strangest coincidence you've ever experienced ?",
    "Have you ever laughed so hard , no sound came out , and you just sat there clapping like a seal ?",
    "Why is it that when you're trying not to laugh , everything becomes ten times funnier ?",
    "Do you think if we didn't have thumbs , we'd still be the dominant species ?",
    "What's the best way to subtly escape a boring conversation ?",
    "Have you ever had a dream where you showed up to work in your pajamas ?",
    "Why do I always crave snacks right after I decide to start eating healthier ?",
    "Do you ever wonder what your pet names you in their head ?",
    "What's the worst haircut you've ever had ?",
    "Have you ever accidentally memorized a song you hate ?",
    "Why is deciding what to have for dinner the hardest decision of the day ?",
    "Is it really a road trip if you don't sing along to the radio at the top of your lungs ?",
    "Have you ever tried to make a recipe you saw in a cartoon ?",
    "What's the funniest thing you've ever seen a stranger do ?",
    "Do you think time travel will ever be possible , and if so , what time would you visit ?",
    "Why is it that every time I try to be cool , I end up doing something awkward ?",
    "If animals could talk , which species do you think would be the most annoying ?"
]


# Greedy Search Decoder

In [6]:
# Create searcher instance
searcher = GreedySearchDecoder(encoder, decoder)

# Input sentence
input_sentence = "how are you ?"

# Evaluate sentence
output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
output_sentence = ' '.join(output_words)

print('Input:', input_sentence)
print('Output:', output_sentence)

Input: how are you ?
Output: i yeah i i i i i i i .


In [7]:
input_utts_normalized = [normalizeString(utt) for utt in input_utts]
input_utts_filtered = [input_utt for input_utt in input_utts_normalized if all(word in voc.word2index.keys() for word in input_utt.split())]
input_utts_filtered[:10]

["why do i always get stuck in the middle of everyone's problems ?",
 'do you think a penguin could be a good pet in the city ?',
 "what's the weirdest thing you've ever done to impress a crush ?",
 "have you ever walked into a room and forgotten why you're there ?",
 "what's your take on eating pizza with a fork and knife ?",
 'is talking to plants really supposed to help them grow ?',
 'i dreamed i was a muffin what do you think it means ?',
 "why is it that i can never find my keys when i'm late ?",
 'do you think ducks have best friends ?',
 "i just found out i've been using a word wrong my entire life ."]

In [8]:
searcher = GreedySearchDecoder(encoder, decoder)
attempts = 50
outputs = [[] for _ in range(len(input_utts_filtered))]

for idx, input_utt in enumerate(input_utts_filtered):
    print(f"Generating responses for input utterance {idx + 1} / {len(input_utts_filtered)}...")
    for _ in range(attempts):
        output_words = evaluate(encoder, decoder, searcher, voc, input_utt)
        output_utt = ' '.join(output_words)
        outputs[idx].append(output_utt)

Generating responses for input utterance 1 / 25...
Generating responses for input utterance 2 / 25...
Generating responses for input utterance 3 / 25...
Generating responses for input utterance 4 / 25...
Generating responses for input utterance 5 / 25...
Generating responses for input utterance 6 / 25...
Generating responses for input utterance 7 / 25...
Generating responses for input utterance 8 / 25...
Generating responses for input utterance 9 / 25...
Generating responses for input utterance 10 / 25...
Generating responses for input utterance 11 / 25...
Generating responses for input utterance 12 / 25...
Generating responses for input utterance 13 / 25...
Generating responses for input utterance 14 / 25...
Generating responses for input utterance 15 / 25...
Generating responses for input utterance 16 / 25...
Generating responses for input utterance 17 / 25...
Generating responses for input utterance 18 / 25...
Generating responses for input utterance 19 / 25...
Generating responses 

In [9]:
distinct_1 = [calculate_distinct_n_grams(outputs[i], n=1) for i in range(len(input_utts_filtered))]
distinct_2 = [calculate_distinct_n_grams(outputs[i], n=2) for i in range(len(input_utts_filtered))]

print(f"Distinct-1: {np.mean(distinct_1):.4f}")
print(f"Distinct-2: {np.mean(distinct_2):.4f}")

Distinct-1: 0.0060
Distinct-2: 0.0082


# Beam Search Decoder

In [ ]:
# Create searcher instance
beam_width = 10
searcher = BeamSearchDecoder(encoder, decoder, beam_width)

# Input sentence
input_sentence = "how are you ?"

# Evaluate sentence
output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
output_sentence = ' '.join(output_words)

print('Input:', input_sentence)
print('Output:', output_sentence)

In [ ]:
input_utts_normalized = [normalizeString(utt) for utt in input_utts]
input_utts_filtered = [input_utt for input_utt in input_utts_normalized if all(word in voc.word2index.keys() for word in input_utt.split())]
input_utts_filtered[:10]

In [ ]:
searcher = GreedySearchDecoder(encoder, decoder)
attempts = 50
outputs = [[] for _ in range(len(input_utts_filtered))]

for idx, input_utt in enumerate(input_utts_filtered):
    print(f"Generating responses for input utterance {idx + 1} / {len(input_utts_filtered)}...")
    for _ in range(attempts):
        output_words = evaluate(encoder, decoder, searcher, voc, input_utt)
        output_utt = ' '.join(output_words)
        outputs[idx].append(output_utt)

In [ ]:
distinct_1 = [calculate_distinct_n_grams(outputs[i], n=1) for i in range(len(input_utts_filtered))]
distinct_2 = [calculate_distinct_n_grams(outputs[i], n=2) for i in range(len(input_utts_filtered))]

print(f"Distinct-1: {np.mean(distinct_1):.4f}")
print(f"Distinct-2: {np.mean(distinct_2):.4f}")